## 云电脑评价情感分类

### 1.数据准备和模型配置

In [ ]:
# 解压ERNIE3.0模型到项目中
!unzip -oq ERNIE.zip

In [ ]:
# 解压基础文本情感分析模型到项目中
import os
os.makedirs("base_model", exist_ok=True)
!unzip -oq base_model.zip -d base_model/

### 2.微调

In [ ]:
cd ERNIE\applications\tasks\text_classification

In [ ]:
# 微调训练
# ernie 中文文本分类模型
# 基于json实现预置网络训练。
!python run_trainer.py --param_path config/cls_ernie_fc_ch.json

### 3.使用基础模型进行预测

In [ ]:
# 创建输出结果文件夹
os.makedirs("output", exist_ok=True)
# 预测, 预测过程中的日志自动保存在output/predict_result_base.txt文件中。
%cd ERNIE/applications/tasks/text_classification
!python run_infer.py --param_path config/cls_ernie_fc_ch_infer_base.json

### 4.使用微调后模型进行预测

In [ ]:
# 预测, 预测过程中的日志自动保存在output/predict_result_finetune.txt文件中。
%cd ERNIE/applications/tasks/text_classification
!python run_infer.py --param_path config/cls_ernie_fc_ch_infer_finetune.json

### 5.微调前后的对比

In [ ]:
txt_base = "output/predict_result_base.txt"
txt_finetune = "output/predict_result_finetune.txt"

with open(txt_base, 'r') as file:
    txt_base_list = []
    for line in file:
        txt_base_list.append(line.strip())  # strip()函数用于去除行末的换行符
with open(txt_finetune, 'r') as file:
    txt_finetune_list = []
    for line in file:
        txt_finetune_list.append(line.strip())  # strip()函数用于去除行末的换行符

# 计算错误类别的得分对比与正确类别的得分对比，这里我们以微调后预测的类别作为正确类别

# 首先得到预测类别的列表，如[0, 1, 0, 0, ...]
label_list = []
for item in txt_finetune_list:
    data = item.split("\t")[1][1:-1].split(", ")
    if float(data[0]) > float(data[1]):
        label_list.append(0)
    else:
        label_list.append(1)
length = len(label_list)

# 统计微调前与微调后错误类别的平均得分以及正确类别的平均得分
base_incorrect, base_correct, finetune_incorrect, finetune_correct = 0, 0, 0, 0
for index in range(length):
    # 微调前的
    item = txt_base_list[index]
    data = item.split("\t")[1][1:-1].split(", ")
    base_incorrect += float(data[1 - label_list[index]])
    base_correct += float(data[label_list[index]])
    
    # 微调后的
    item = txt_finetune_list[index]
    data = item.split("\t")[1][1:-1].split(", ")
    finetune_incorrect += float(data[1 - label_list[index]])
    finetune_correct += float(data[label_list[index]])

base_incorrect /= length
base_correct /= length
finetune_incorrect /= length
finetune_correct /= length

print("错误类别平均得分，" + "微调前：", base_incorrect, "微调后：", finetune_incorrect)
print("正确类别平均得分，" + "微调前：", base_correct, "微调后：", finetune_correct)